# Исследование надёжности заёмщиков

***Заказчик*** — кредитный отдел банка.

***Поставленная задача*** - провести исследование статистики о платежеспособности клиентов банка и ***ответить на вопросы***:

1. "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
2. "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
3. "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
4. "Как разные цели кредита влияют на его возврат в срок?".

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

***План*** выполнения работы:
1. Изучение общей информации о данных;
2. Предобработка данных;
3. Исследование вопросов задачи;
4. Подготовка общего вывода.

## Изучение общей информации о данных

In [1]:
# импорт библтотеки pandas
import pandas as pd

# чтение файла с данными с сохранением в 'data'
data = pd.read_csv('/datasets/data.csv')

# получение первых 10 строк таблицы 'data'
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [2]:
# Получение общей информации о данных в датасете
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Рассмотрим полученную информацию подробнее.

Всего в таблице 12 столбцов со следующими типами данных: *int64, float64 и object*.

Подробно разберём, какие в датасете столбцы и какую информацию они содержат:

- ***children*** — количество детей в семье;
- ***days_employed*** — общий трудовой стаж в днях; 
- ***dob_years*** — возраст клиента в годах; 
- ***education*** — уровень образования клиента; 
- ***education_id*** — идентификатор уровня образования; 
- ***family_status*** — семейное положение; 
- ***family_status_id*** — идентификатор семейного положения; 
- ***gender*** — пол клиента; 
- ***income_type*** — тип занятости; 
- ***debt*** — имелась ли задолженность по возврату кредитов; 
- ***total_income*** — ежемесячный доход; 
- ***purpose*** — цель получения кредита.

Количества значений в столбцах ***days_employed*** и ***total_income*** отличаются от количеств значений в других столбцах. Это говорит о том, что в данных столбцах есть отсутствующие значения.

В столбце ***days_employed*** содержатся, в основном, отрицательные значения. Встречающиеся в столбце положительные значения на порядок отличаются от основной массы значений без учета знака.

**Вывод**

Каждая строка таблицы содержит информацию о возрасте, поле, семейном положении и наличии детей у заемщиков, а также об их общем трудовом стаже, типе занятости, ежемесяном доходе, наличию долгов и цели получения кредита. 

Для определения влияния семейного положения и количества детей клиента на факт погашения кредита в срок особенно ценны столбцы ***children***, ***family_status*** и ***debt***.

Во избежание искажения результатов дальнейшего анализа рассмотренной таблицы необходимо решить проблему с наличием пропусков в столбцах ***days_employed*** и ***total_income***.

Также имеется проблема с представлением данных в столбце ***days_employed***, решение которой не приоритетно в виду того, что данный столбец не является ключевым для решения поставленной задачи.

## Предобработка данных

### Обработка пропусков и артефактов

Количества пропущенных значений в столбцах ***days_employed*** и ***total_income*** равны, что может говорить о неслучайном характере возникновения пропусков и быть связано с типом занятости заемщиков.

Проверим данное предположение:

In [3]:
# вывод перывых 10 строк, содержащих пропущенные значения в столбцах days_employed и total_income
data.loc[(pd.isna(data.loc[:,'days_employed'])) & (pd.isna(data.loc[:,'total_income']))].head(10) 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


Из полученной таблицы видно, что пропуски в столбцах ***days_employed*** и ***total_income*** не связаны с каким-либо конкретным показателем из других столбцов таблицы.

Так как строки, в которых имеются пропуски, тем не менее, содержат ценную информацию, наиболее оптимальным методом обработки пропусков в данном случае будет являться их замена на усредненные значения.

Для корректного дальнейшего расчета в столбце ***days_employed*** необходимо выявить все артефакты и обработать их.

In [4]:
# Пересчет значений столбца days_employed в годы для лучшего понимания данных
years_employed = data['days_employed'] / 365

# Визуальный осмотр полученных значений
print(years_employed.head(10))
print()
print(years_employed.tail(10))

0    -23.116912
1    -11.026860
2    -15.406637
3    -11.300677
4    932.235814
5     -2.537495
6     -7.888225
7     -0.418574
8    -18.985932
9     -5.996593
Name: days_employed, dtype: float64

21515      -1.281329
21516      -2.505182
21517      -1.108710
21518    1024.645783
21519      -6.442279
21520     -12.409087
21521     942.294258
21522      -5.789991
21523      -8.527347
21524      -5.437007
Name: days_employed, dtype: float64


Из первых и поcледних 10 строк полученной таблицы видно, что отрицательные значения без учета знака отражают реальный стаж заемщиков, а положительные данные являются выбросами.

Рассмотрим строки таблицы, соответствующие положительным значениям в столбце ***days_employed*** более подробно для понимания закономерности появления выбросов:

In [5]:
# вывод строк таблицы 'data', содержащих положительные значения в столбце 'days_employed'
data.loc[data.loc[:,'days_employed'] > 0]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
18,0,400281.136913,53,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823.777243,на покупку подержанного автомобиля
24,1,338551.952911,57,среднее,1,Не женат / не замужем,4,F,пенсионер,0,290547.235997,операции с коммерческой недвижимостью
25,0,363548.489348,67,среднее,1,женат / замужем,0,M,пенсионер,0,55112.757732,покупка недвижимости
30,1,335581.668515,62,среднее,1,женат / замужем,0,F,пенсионер,0,171456.067993,операции с коммерческой недвижимостью
...,...,...,...,...,...,...,...,...,...,...,...,...
21505,0,338904.866406,53,среднее,1,гражданский брак,1,M,пенсионер,0,75439.993167,сыграть свадьбу
21508,0,386497.714078,62,среднее,1,женат / замужем,0,M,пенсионер,0,72638.590915,недвижимость
21509,0,362161.054124,59,высшее,0,женат / замужем,0,M,пенсионер,0,73029.059379,операции с недвижимостью
21518,0,373995.710838,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем


Из полученной таблицы видно, что выбросы в столбце ***days_employed*** связаны с типом занятости ***пенсионер*** в столбце ***income_type***.

Тем не менее, логика образования данных значений не ясна, и, не имея возможности обращения к специалистам, осуществлявшим выгрузку, исправить данные значения на реальные не представляется возможным.

Выбросы, а также пропущенные значения в столбце ***days_employed*** можно заменить на расчетные значения, полученные с помощью корректных значений в столбцах ***dob_years*** и ***days_employed***. С помощью имеющихся данных в указанных столбцах можно рассчитать коэффициент накопления стажа, отражающего количество дней стажа, приходящихся на 1 год возраста человека. Назовем данный коэффициент ***days_employed_per_age***, произведем расчеты и замену артефактов и пропущенных значений в столбце ***days_employed***.

In [6]:
# Сохранение строк датасета, соответствующих корректным значениям столбца 'days_employed', в переменной 'real_exp'
real_exp = data.loc[data.loc[:, 'days_employed'] < 0]

# Расчет коэффициента накопления стажа
days_employed_per_age = real_exp['days_employed'].sum() / real_exp['dob_years'].sum()

# Создание функции для пересчета артефактов столбца 'days_employed'
def recount_artifacts(row):
    # Пересчет положительных значений
    if row['days_employed'] > 0:
        recounted = row['dob_years'] * days_employed_per_age
        return recounted
    else:
    # Пересчет пустых значений
        if pd.isna(row['days_employed']) == True:
            recounted = row['dob_years'] * days_employed_per_age
            return recounted
        else:
            return row['days_employed']
    
# Применение функций пересчета к стобцу 'days_employed'
data['days_employed'] = data.apply(recount_artifacts, axis = 1)
# Вывод первых и последних 10 строк таблицы 'data' для проверки замены положительных значений в столбце 'days_employed'
display(data.head(10))
display(data.tail(10))

# Получение общей информации о датасете
data.info()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,-3131.977448,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21515,1,-467.685130,28,среднее,1,женат / замужем,0,F,сотрудник,1,109486.327999,заняться образованием
21516,0,-914.391429,42,высшее,0,женат / замужем,0,F,компаньон,0,322807.776603,покупка своего жилья
21517,0,-404.679034,42,высшее,0,гражданский брак,1,F,компаньон,0,178059.553491,на покупку своего автомобиля
21518,0,-3486.540933,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем
21519,1,-2351.431934,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949.039788,покупка коммерческой недвижимости
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,-3959.292246,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля
21524,2,-1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Исходя из первых и последних 10 строк датасета, а также общей информации о данных после преобразований столбца ***days_employed*** можно судить об успешной замене выбросов и пропущенных значений.

Далее произведем замену пропущенных значений в столбце ***total_income***, значениями, рассчитанными как медианы значений столбца ***total_income***, соответствующих каждому типу занятости в столбце ***income_type***:

In [7]:
# Определение уникальных типов занятости в столбце 'income_type'
data['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

In [8]:
#Создание функции для замены пропусков в столбце 'total_income' медианами значений столбца 'total_income', 
 #соответствующих каждому типу занятости в столбце 'income_type' и уровню образования в столбце 'education'
def income_type_fillna(inc_type, education):
    filtered_data = data.loc[
        (data.loc[:, 'income_type'] == inc_type) &
        (data.loc[:, 'education'] == education)
    ]
    median = filtered_data['total_income'].median()
    
    data.loc[
        (data.loc[:, 'income_type'] == inc_type) &
        (data.loc[:, 'education'] == education)] = data.loc[
        (data.loc[:, 'income_type'] == inc_type) &
        (data.loc[:, 'education'] == education)].fillna(value = median)
    
# Задание списков уникальных значений столбцов 'education' и 'income_type' соответствубщим переменным
income_type_list = data['income_type'].unique()
education_list = data['education'].unique()

# Создание функции для автоматической подстановки уникальных значений столбцов 'education' и 'income_type' 
# в функцию income_type_fillna 

def faster_fillna(income_type_list, education_list):
    for income_type in income_type_list:
        for education_type in education_list:
            income_type_fillna(income_type, education_type)

# Замена пустых значений с помощью созданной функции замены пропусков

faster_fillna(income_type_list, education_list)

# Получение общей информации о таблице 'data'
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Исходя из полученной информации об обновленном датасете, пропуски в столбце ***total_income*** успешно заменены

**Вывод**

На этапе обработки пропусков в одних и тех же строках столбцов ***days_employed*** и ***total_income*** были обнаружены пропуски данных. Данные пропуски не связаны с данными других столбцов, и их наличие в выгрузке может быть связано с неполным внесением данных о заемщиках в базу данных.

Кроме пустых значений, в столбце ***days_employed*** были обнаружены выбросы в виде положительных значений высокого порядка. Данные выбросы связаны непосредственно с типом занятости ***пенсионер***, что может быть связано с некорректным расчетом полного трудового стажа для данного типа занятости заемщиков.

Пустые значения и выбросы в столбце ***days_employed*** были заменены на расчетные значения, вычисленные с помощью имеющихся корректных данных о возрасте заемщиков и полном трудовом стаже.

Пустые значения в столбце ***total_income*** были заменены на медианы ежемесячного дохода заемщиков разного типа занятости.

### Замена типа данных

С целью улучшения визуального представления данных в датасете, а также для удобства дальнейшей работы с данными произведем замену значений и типов данных столбцов ***days_employed*** и ***total_income***:

In [9]:
# Замена отрицательных значений столбца 'days_employed' на положительные
data['days_employed'] = data['days_employed'].abs()

# Изменение типа данных столбца 'days_employed' на int, для отображения в таблице целых значений
data['days_employed'] = data['days_employed'].astype('int')

# Изменение типа данных столбца 'total_income' на int, для отображения в таблице целых значений
data['total_income'] = data['total_income'].astype('int')

data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,3131,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


**Вывод**

C целью улучшения визуального представления данных, типы данных столбцов ***days_employed*** и ***total_income*** заменены с ***float*** на ***int*** с помощью метода *.astype( )*.

### Обработка дубликатов

Выведем на экран количества уникальных значений столбцов ***children*, *education*, *family_status*, *gender*, *income_type*, *dept* и *purpose*** c целью поиска неявных дубликатов:

In [10]:
# Создание функции для вывода на экран уникальных значений столбцов
def print_uniques(column_name):
    print(f'Уникальные значения столбца {column_name}')
    print(data[column_name].value_counts())
    print()
    
# Вывод на экран уникальных значений столбцов 
print_uniques('children')
print_uniques('education')
print_uniques('family_status')
print_uniques('gender')
print_uniques('income_type')
print_uniques('debt')
print_uniques('purpose')


Уникальные значения столбца children
 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Уникальные значения столбца education
среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

Уникальные значения столбца family_status
женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

Уникальные значения столбца gender
F      14236
M       7288
XNA        1
Name: gender, dtype

В результате рассмотрения уникальных значений перечисленных столбцов, были выявлены одинаковые значения в столбце ***education***, отличающиеся регистрами.

Кроме того, в столбце ***children*** были обнаружены выбросы в виде значений 20 и -1.

Данные выбросы могут быть связаны с некорректным переносом данных в электронную базу данных. При дальнейшей обработке данные значения будут заменены на 2 и 1 соответственно.

Приведем значения в столбце ***education*** к нижнему регистру, чтобы учесть все дубликаты при дальнейшей обработке.

In [11]:
# Перевод значений столбца education в нижний регистр
data['education'] = data['education'].str.lower()

# Проверка списка уникальных значения столбца 'education' после замены регистра
print(data['education'].value_counts())

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64


В результате обработки значений столбца ***education*** количество уникальных значений было уменьшено до количества уровней образования, имеющихся в таблице.

Заменим выбросы в столбце ***children*** на более логичные значения:

In [12]:
# Замена выбросов
data['children'] = data['children'].replace(-1,1)
data['children'] = data['children'].replace(20,2)

# Проверка замены выбросов
data['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

В результате обработки значений столбца ***children*** выбросы были успешно заменены.

Рассчитаем количество грубых дубликатов строк в датасете и удалим их:

In [13]:
# Расчет суммы дубликатов строк в датасете
data.duplicated().sum()

54

In [14]:
# Удаление полных дубликатов строк в датасете
data = data.drop_duplicates().reset_index(drop = True)
# Проверка общей информации о датасете
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21471 entries, 0 to 21470
Data columns (total 12 columns):
children            21471 non-null int64
days_employed       21471 non-null int64
dob_years           21471 non-null int64
education           21471 non-null object
education_id        21471 non-null int64
family_status       21471 non-null object
family_status_id    21471 non-null int64
gender              21471 non-null object
income_type         21471 non-null object
debt                21471 non-null int64
total_income        21471 non-null int64
purpose             21471 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

На этапе обработки дубликатов в столбце ***education*** были обнаружены одинаковые значения, записанные в разном регистре. Для обработки данных значений был использован метод *str.lower()*.

Причиной отражения данных в разном регистре может являться ручной ввод данных о заемщиках.

В столбце ***purpose*** были обнаружены неявные дубликаты, подходящие под разделение на категории, работа над которыми будет проводиться в разделах ***2.4 Лемматизация*** и ***2.5 Категоризация данных***.

Причиной наличия неявных дубликатов может является ручной ввод данных о заемщиках.

Поcле обработки данных в столбце ***education*** были произведены поиск и удаление грубых дубликатов строк датасета c помощью методов *duplicated() и drop_duplicates().

Причиной наличия грубых дубликатов строк может являться некорректная выгрузка данных.

### Лемматизация

Для последующего разделения данных в столбце ***purpose*** по категориям необходимо лемматизировать столбец с целью получения удобных для обработки форм слов, составляющих цели.

In [15]:
# Импорт библиотеки pymystem3
from pymystem3 import Mystem
m = Mystem()

# Задание переменной для сохранения лемм столбца *purpose*
lemmas = []

# Лемматизация
for purpose in data['purpose']:
    lemma = ' '.join(m.lemmatize(purpose))
    lemmas.append(lemma)

# Создание в датасете столбца для сохранения данных  переменной 'lemmas' и последующей категоризации данных
data['purpose_category'] = lemmas

# Вывод первых 5 строк столбцов 'purpose' и 'purpose_category'
data.loc[:, ['purpose', 'purpose_category']].head()


,purpose,purpose_category
0,покупка жилья,покупка жилье \n
1,приобретение автомобиля,приобретение автомобиль \n
2,покупка жилья,покупка жилье \n
3,дополнительное образование,дополнительный образование \n
4,сыграть свадьбу,сыграть свадьба \n


**Вывод**

С целью дальнейшей обработки и категоризации списка целей получения кредита, была проведена лемматизация столбца ***purpose*** c примененем функционала библиотеки *pymystem3*.

Полученные леммы были сохранены в столбце ***purpose_categories*** для дальнейшей обработки.

### Категоризация данных

Определим категории целей получения кредита из полученных лемм.

In [16]:
# Вывод списка уникальных значений из столбца 'purpose_category'
data['purpose_category'].value_counts()

автомобиль \n                                    972
свадьба \n                                       793
на   проведение   свадьба \n                     773
сыграть   свадьба \n                             769
операция   с   недвижимость \n                   675
покупка   коммерческий   недвижимость \n         662
операция   с   жилье \n                          652
покупка   жилье   для   сдача \n                 652
операция   с   коммерческий   недвижимость \n    650
покупка   жилье \n                               646
жилье \n                                         646
покупка   жилье   для   семья \n                 638
строительство   собственный   недвижимость \n    635
недвижимость \n                                  633
операция   со   свой   недвижимость \n           627
строительство   жилой   недвижимость \n          625
покупка   недвижимость \n                        621
покупка   свой   жилье \n                        620
строительство   недвижимость \n               

In [17]:
# Определение необходимых для дальнейшего анализа категорий и присвоение их соответствующим переменным
auto = 'автомобиль'
wedding = 'свадьба'
realty = 'недвижимость'
accommodation = 'жилье'
education = 'образование'

# Создание функции для замены лемм на соответствующие категориии
def categorize_purpose(category):
    for lemm in data['purpose_category']:
        if category in lemm:
            data['purpose_category'] = data['purpose_category'].replace(lemm, category)
            
# Замена лемм на соответcтвующие категории
categorize_purpose(auto)
categorize_purpose(wedding)
categorize_purpose(realty)
categorize_purpose(accommodation)
categorize_purpose(education)

# Создание функции для объединения категорий 'недвижимость' и 'жилье' в категорию 'недвижимость'
data['purpose_category'] = data['purpose_category'].replace('жилье','недвижимость')
data['purpose_category'].value_counts()

недвижимость    10814
автомобиль       4308
образование      4014
свадьба          2335
Name: purpose_category, dtype: int64

Для предстоящей оценки связи между уровнем дохода заемщиков и возвратом кредитов в срок, необходимо разделить доходы заемщиков на категории, так как список уникальных значений в столбце ***income_total*** достаточно обширный, а работа с единичными значениями не позволит сделать из них статистические выводы.

In [18]:
# Вывод описательной статистики столбца *total_income*
data['total_income'].describe()

count    2.147100e+04
mean     1.654486e+05
std      9.827101e+04
min      2.066700e+04
25%      1.075200e+05
50%      1.432060e+05
75%      1.982515e+05
max      2.265604e+06
Name: total_income, dtype: float64

In [19]:
# Создание функци для присвоения категорий строкам столбца 'income_category', соответствующих доходу каждого заемщика
def categorize_income(income):
# Выделение категорий заработка на основании описательной статистики и присвоение их соответствующим переменным
    low = '< 110000'
    below_average = '110000 - 140000'
    above_average = '140000 - 195000'
    high = '> 195000'
    
# Задание условий присвоения категорий
    if income < 110000:
        return low
    if 110000 <= income < 140000:
        return below_average
    if 140000 <= income < 195000:
        return above_average
    if income >= 195000:
        return high

# Присвоение назначенных категорий строкам столбца 'income_category'
data['income_category'] = data['total_income'].apply(categorize_income)

# Вывод первых 5 строк столбцов 'total_income' и 'income_category'
data.loc[:, ['total_income', 'income_category']].head()

,total_income,income_category
0,253875,> 195000
1,112080,110000 - 140000
2,145885,140000 - 195000
3,267628,> 195000
4,158616,140000 - 195000


Из полученной таблицы видно, что категоризация прошла успешно.

**Вывод**

На этапе категоризации данных были выделены ***5 категорий целей получения кредита***:

- недвижимость;
- автомобиль;
- образование;
- свадьба.

Категоризация целей получения кредита была проведена на основании лемматизированных значений столбца ***purpose*** c последующей заменой лемм на соответствующие категории, определенные из уникальных значений списка целей.

Также были выделены ***4 категории ежемесячного дохода заемщиков***:

- до 110000;
- от 110000 до 140000;
- от 140000 до 195000;
- свыше 195000.

Категоризация ежемесячного дохода была проведена на основании описательной статистики столбца ***total_income***. За крайние точки диапазонов категорий были взяты 25-й, 50-й, 75-й процентили из диапазона значений столбца.

## Подготовка ответов на вопросы

Для определения влияния различных показателей на наличие задолженностей по кредитам необходимо учитывать количество человек, характеризуемых данными показателями, и определять долю невозврата в рассматриваемой выборке.

Например, заемщиков, не имеющих детей, в разы больше, чем заемщиков, имеющих хотя бы одного ребенка. Таким образом, и количество долгов у большей категории заемщиков будет выше. В этой ситуации абсолютные значения суммы количества случаев задолженности в рассматриваемых категориях не позволят сделать правильный вывод о влиянии количества детей заемщиков на возврат ими кредитов в срок.

Напишем функцию, которая позволит правильно ответить на поставленные в задаче вопросы за счет учета доли невозврата кредитов заемщиками в определенных выборках:

In [21]:
# Создание функции для формирования и вывода на экран сводной таблицы, рассчитывающей доли невозврата кредитов в срок
 # где influencing_indicator - влияющий показатель
 # где dependent_indicator - зависимый показатель
def indicators_relation(influencing_indicator,dependent_indicator):
    # Создание сводной таблицы
    data_pivot_result = data.pivot_table(index=[influencing_indicator],values=dependent_indicator, aggfunc='mean')
    # Вывод сводной талицы на экран
    display(data_pivot_result.sort_values(by = dependent_indicator, ascending = False))

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [22]:
# Рассмотрим связь между наличием детей у заемщиков и долей невозврата кредитов в срок
indicators_relation('children','debt')

,debt
children,
4,0.097561
2,0.094925
1,0.091639
3,0.081818
0,0.075353
5,0.000000


**Вывод**

Результаты расчетов показали, что ***зависимости между количеством детей у заемщиков и возвратом кредитов в срок нет***. Судя по всему, как ответственными, так и безответственными людьми могут быть как родители, так и люди, не имеющие детей. Кроме того, вероятно, не только уровень ответственности заемщиков играет роль в своевременном возврате кредитов.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [25]:
# Рассмотрим связь между семейным положением заемщиков и долей невозврата кредитов в срок
indicators_relation('family_status','debt')

,debt
family_status,
Не женат / не замужем,0.097509
гражданский брак,0.093202
женат / замужем,0.075421
в разводе,0.071130
вдовец / вдова,0.065693


**Вывод**

Согласно приведенной таблице, ***заемщиками, наиболее способными вернуть кредит в срок, являются люди, которые есть или были в законном браке***.

Наименее надежными категориями заемщиков являются неженатые/незамужние люди и люди в гражданском браке. 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [26]:
# Рассмотрим связь между уровнем дохода и количествам задолженностей по кредитам
indicators_relation('income_category','debt')

,debt
income_category,
110000 - 140000,0.087487
140000 - 195000,0.086658
< 110000,0.080517
> 195000,0.070817


**Вывод**

Результаты расчетов показали, что ***зависимости между уровнем дохода заемщиков и возвратом кредитов в срок нет***. Отсутствие выраженной зависимости между рассмотренными показателями может быть связано с размером семей заемщиков, где более показательным был бы уровень дохода на человека в семье.

- Как разные цели кредита влияют на его возврат в срок?

In [27]:
# Рассмотрим связь между целями кредита и количествам задолженностей по кредитам
indicators_relation('purpose_category','debt')

,debt
purpose_category,
автомобиль,0.093547
образование,0.092177
свадьба,0.079657
недвижимость,0.072314


**Вывод**

Результаты расчетов показывают, что ***наиболее надежными заемщиками являются люди, оформляющие кредиты на операции с недвижимостью***. Предположительно, это может быть связано с наибольшей подготовленностью данной категории заемщиков к шагу оформления кредита как морально, так и материально.

## Общий вывод

В ходе анализа данных о платежеспособности клиентов банка было рассмотрено влияние следующих показателей на возврат кредита в срок:

- Количество детей заемщика;
- Семейное положение заемщика;
- Уровень дохода заемщика;
- Цель кредита.

В результате рассмотрения каждого из этих показателей в отдельности на возврат заемщиком кредита в срок не было выявлено выраженных зависимостей. Выводы, полученные по результатам каждого из расчетов, основываются больше на предположениях или новых гипотезах, которые необходимо подтверждать.

Тем не менее, в результате анализа вышеперечисленных показателей, были получены следующие результаты:

- ***Самыми надежными*** заемщиками являются:

  1. Вдовцы / вдовы (процент невозврата кредита = 6,57%)
  2. Люди, с ежемесячным доходом более 195 000 рублей (процент невозврата кредита = 7,08%);
  3. Люди, оформляющие кредиты на операции с недвижимостью (процент невозврата кредита = 7,2%)
  4. Бездетные люди (процент невозврата кредита = 7,54%)
 
 
- ***Самыми ненадежными*** заемщиками являются:

  1. Люди с четырьмя детьми (процент невозврата кредита = 9,76%);
  2. Неженатые / незамужние люди (процент невозврата кредита = 9,75%);
  3. Люди, оформляющие кредиты на покупку автомобиля (процент невозврата кредита = 9,35%)
  4. Люди, с ежемесячным доходом от 110 до 140 тыс. рублей (процент невозврата кредита = 8,75%);


Помимо влияния рассмотренных показателей на возврат кредитов в срок, не было рассмотрено влияние стажа, возраста, образования и типа занятости заемщиков, информация о которых имеется в данных о платежеспособности клиентов.

Вероятнее всего, каждый из этих показателей в отдельности не даст достаточно точного ответа на вопрос "Что влияет на погашение заемщиками кредитов в срок?". Для поиска ответа на этот вопрос необходимо рассмотрение влияния всех перечисленных показатетелей в комплексе.